In [2]:
# Packages for data processing and machine learning
import numpy as np, pandas as pd, sklearn as sk, torch as th 
# Packages for webpage crawling
import requests as r
from bs4 import BeautifulSoup as BS
# Packages for nature language processing
import spacy, pyinflect
from pyinflect import getAllInflections
import nltk,emoji,string,re
from nltk.corpus import stopwords as sw
import gender_guesser.detector as gen
# Packages for Twitter API and configuration
import tweepy as tw, configparser  
# Packages about time
import time as t, datetime as dt, rfc3339
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings('ignore')


c:\Users\ky002\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [163]:
# Reqest the webpages including the synonymsous words or meanings of these topics: car, buy, and drive
car_url='https://www.thesaurus.com/browse/car'
buy_url='https://www.thesaurus.com/browse/buy'
buying_url='https://www.thesaurus.com/browse/buying'
drive_url='https://www.thesaurus.com/browse/drive'

# Crawling the synonyms of four topics: (car, buy, buying, drive) from Thesurus.com
def keyword_extract(url_name,class_name):
    page_name=r.get(url_name)
    soup_name=BS(page_name.content,'html.parser')
    key_soup=soup_name.find('ul', class_=class_name).find_all('a')
    list_name=[]
    for key in key_soup:
        new_key=key['href'][8:].replace('%20',' ')
        list_name.append(new_key)
    return list_name

# I only captured the red-marked words which contain the most closed meanings as the chosen topics
car_sym=keyword_extract(car_url,'css-1xohnkh e1ccqdb60')[:14]
car_sym.insert(0,'car')
buy_sym=keyword_extract(buy_url,'css-wmtunb e1ccqdb60')[:4]
buy_sym.insert(0,'buy')
buying_sym=keyword_extract(buying_url,'css-1lj4erq e1ccqdb60')[:3]
buying_sym.insert(0,'buying')
buy_sym=buy_sym+buying_sym
drive_sym=keyword_extract(drive_url,'css-n85ndd e1ccqdb60')[:4]
drive_sym.insert(0,'drive')

# Reqest the webpages including the synonymsous words or meanings of these topics: car, buy, and drive
car_url='https://www.thesaurus.com/browse/car'
buy_url='https://www.thesaurus.com/browse/buy'
buying_url='https://www.thesaurus.com/browse/buying'
drive_url='https://www.thesaurus.com/browse/drive'

# Crawling the synonyms of four topics: (car, buy, buying, drive) from Thesurus.com
def keyword_extract(url_name,class_name):
    page_name=r.get(url_name)
    soup_name=BS(page_name.content,'html.parser')
    key_soup=soup_name.find('ul', class_=class_name).find_all('a')
    list_name=[]
    for key in key_soup:
        new_key=key['href'][8:].replace('%20',' ')
        list_name.append(new_key)
    return list_name

# I only captured the red-marked words which contain the most closed meanings as the chosen topics
car_sym=keyword_extract(car_url,'css-1xohnkh e1ccqdb60')[:14]
car_sym.insert(0,'car')
buy_sym=keyword_extract(buy_url,'css-wmtunb e1ccqdb60')[:4]
buy_sym.insert(0,'buy')
buying_sym=keyword_extract(buying_url,'css-1lj4erq e1ccqdb60')[:3]
buying_sym.insert(0,'buying')
buy_sym=buy_sym+buying_sym
drive_sym=keyword_extract(drive_url,'css-n85ndd e1ccqdb60')[:4]
drive_sym.insert(0,'drive')

# Altough I have got thirty words as required keywords for tweet requests, it is recommended to include different tenses or forms of words 
#, as many Twitter users will use them depending on different contents.
nlp = spacy.load('en_core_web_sm')

buy_str=' '.join(buy_sym)
buy_doc=nlp(buy_str)
buy_extension_list=[]
for num in range(len(buy_doc)):
    token = buy_doc[num]
    if token.tag_ in ['NN','VB','VBG']:
        buy_extension_list.append(token._.inflect('VB',inflect_oov=True))
buy_extension_list=[ele for ele in list(set(buy_extension_list+['invest','shop','transact'])) if ele]

def extension(sym):
    sym_str=' '.join(sym)
    sym_token=nlp(sym_str)
    extension_list=[]
    for num in range(len(sym_token)):
        token = sym_token[num]
        if token.tag_ in ['NN','VB','VBG']:
            if str(token)!=token._.inflect('VBD',inflect_oov=True)[:len(token)]:
                extension_list.append(token._.inflect('VBD',inflect_oov=True))              
            if str(token)!=token._.inflect('VBG',inflect_oov=True)[:len(token)]:
                 extension_list.append(token._.inflect('VBG',inflect_oov=True))
            if str(token)!=token._.inflect('VBN',inflect_oov=True)[:len(token)]:
                 extension_list.append(token._.inflect('VBN',inflect_oov=True))
            if str(token)!=token._.inflect('VBZ',inflect_oov=True)[:len(token)]:
                 extension_list.append(token._.inflect('VBZ',inflect_oov=True))
    return extension_list

buy_sym=list(set(extension(buy_extension_list)+buy_extension_list))
buy_sym.append('acquisition')

drive_sym=list(set(extension(drive_sym)+drive_sym))

# According to car registration records of the UK from 2019 to 2022, the top 10 sales brands were selected as they might be mentioned more frequently than the other brands when talking about automative topics
brand_list=['Ford','BMW','Volkswagen','Mercedes-Benz','Audi','Vauxhall','Toyota','Kia','Hyundai','Land Rover']

#While searching for Google Trends data, various spellings or expressions that a user could use while posting about a car model were checked.
brand_abb=['VW','Mercedes','Voho','Landy','Bimmer','MBZ']

# The most popular Car sale Models in the UK, as a same replacement of the brand name: One model for one brand
brand_model=['fiesta','corolla','Series','polo','sportage','tucson','corsa','A-Class','discovery','A3']

# Name of car types when people are talking a series of car
car_type=['coupe','hatchback','sedan','sports','suv']

# Names of many important parts of car are also added in as I found that is a part of comments when people value their cars:15 most important words from https://www.collinsdictionary.com/word-lists/car-parts-of-a-car
car_tool=['grip','bumper','tyre','brake','bonnet','airbag','carburettor','piston','engine','battery','fuel tank','hood','steering wheel','accelerator','seatbelt']
#But there isn't such words like automobiling/automobiled or jeeping/jeeped, so it has to be deleted
car_sym.remove('ride')
car_sym.append('vehicle')

def df_generator(name,list):
    return pd.DataFrame({name:list})
#Put all the words into a dataframe sorted by different segements.
keywords=pd.concat([df_generator('Car',car_sym),
                    df_generator('Buy',buy_sym),
                    df_generator('Drive',drive_sym),
                    df_generator('Brand',brand_list),
                    df_generator('Brand Abbreviation',brand_abb),
                    df_generator('Brand Model',brand_model), 
                    df_generator('Car Tool',car_tool),
                    df_generator('Car Type',car_type)],
                    axis=1)
print(keywords)

# To limit the search zone that has to contain the elements of car, purchase, drive 
keylist_of_car=car_sym+brand_list+brand_abb+brand_model+car_tool+car_type
keylist_of_buy=buy_sym
keylist_of_drive=drive_sym

#Build the content of query: element 'car' is compulsory which the other elements are optional
query_content='('+' '.join(keylist_of_car).replace(' ',' OR ')+') ('+' '.join(keylist_of_buy+keylist_of_drive).replace(' ',' OR ')+') lang:en place_country:GB -is:nullcast -has:links'
print(len(query_content), query_content)
# Build and read the config for password safety
config=configparser.RawConfigParser()
config.read('config.ini')

api_key=config['twitter']['api_key']
api_key_secret=config['twitter']['api_key_secret']
access_token=config['twitter']['access_token']
access_token_secret=config['twitter']['access_token_secret']
bearer_token=config['twitter']['bearer_token']

# Authtication of my Twitter api
client = tw.Client(bearer_token,api_key, api_key_secret,access_token, access_token_secret,wait_on_rate_limit=True)

# Building the timestrap of each hour
def date_range(start_date, end_date):
    while start_date <= end_date:
        yield start_date
        start_date+=dt.timedelta(hours=8)

# Set the starting at the first hour of Feburary 1st 2019 and the end time at the same hour of October 1st 2021
start_date_1_1 = datetime(2019, 2, 1, 0, 00,00)
end_date_1_1 = datetime(2019, 4, 1, 0, 00,00)
start_date_1_2 = datetime(2019, 8, 1, 0, 00,00)
end_date_1_2 = datetime(2019, 10, 1, 0, 00,00)

start_date_2_1 = datetime(2020, 2, 1, 0, 00,00)
end_date_2_1 = datetime(2020, 4, 1, 0, 00,00)
start_date_2_2 = datetime(2020, 8, 1, 0, 00,00)
end_date_2_2 = datetime(2020, 10, 1, 0, 00,00)

start_date_3_1 = datetime(2021, 2, 1, 0, 00,00)
end_date_3_1 = datetime(2021, 4, 1, 0, 00,00)
start_date_3_2 = datetime(2021, 8, 1, 0, 00,00)
end_date_3_2 = datetime(2021, 10, 1, 0, 00,00)

# remember to change about the time

def time_builder(start_date,end_date):
    time=[]
    for single_date in date_range(start_date, end_date):
        time.append(single_date.strftime("%Y-%m-%d %H:%M:%S"))
    return time

first_time_1=time_builder(start_date_1_1,end_date_1_1)[:-1]+time_builder(start_date_1_2,end_date_1_2)[:-1]
second_time_1=time_builder(start_date_1_1,end_date_1_1)[1:]+time_builder(start_date_1_2,end_date_1_2)[1:]

first_time_2=time_builder(start_date_2_1,end_date_2_1)[:-1]+time_builder(start_date_2_2,end_date_2_2)[:-1]
second_time_2=time_builder(start_date_2_1,end_date_2_1)[1:]+time_builder(start_date_2_2,end_date_2_2)[1:]

first_time_3=time_builder(start_date_3_1,end_date_3_1)[:-1]+time_builder(start_date_3_2,end_date_3_2)[:-1]
second_time_3=time_builder(start_date_3_1,end_date_3_1)[1:]+time_builder(start_date_3_2,end_date_3_2)[1:]

def rfc_time_convetor(time_list):
    new_time_list=[]
    for single_record in time_list:
        datetime_object = datetime.strptime(single_record, "%Y-%m-%d %H:%M:%S")
        rfc_records=rfc3339.rfc3339(datetime_object)
        new_time_list.append(rfc_records)
    return new_time_list

rfc_first_time=rfc_time_convetor(first_time_1)+rfc_time_convetor(first_time_2)+rfc_time_convetor(first_time_3)
rfc_second_time=rfc_time_convetor(second_time_1)+rfc_time_convetor(second_time_2)+rfc_time_convetor(second_time_3)

              Car          Buy    Drive          Brand Brand Abbreviation  \
0             car     transact      run           Ford                 VW   
1            auto      acquire  driving            BMW           Mercedes   
2      automobile          buy     rode     Volkswagen               Voho   
3             bus      bargain     ride  Mercedes-Benz              Landy   
4     convertible     purchase    drove           Audi             Bimmer   
5            jeep         shop   riding       Vauxhall                MBZ   
6       limousine   purchasing     tour         Toyota                NaN   
7         machine    acquiring     trip            Kia                NaN   
8           motor       invest    drive        Hyundai                NaN   
9          pickup  acquisition   ridden     Land Rover                NaN   
10  station wagon          NaN      NaN            NaN                NaN   
11          truck          NaN      NaN            NaN                NaN   

In [ ]:
# Retrieve all the required data from Twitter API from each eight hours per day of three years
for start_time, end_time in zip(rfc_first_time,rfc_second_time):
    tweet_info_small_list=[]
    tweets=client.search_all_tweets(
                            query_content,                            
                            end_time=end_time,       
                            start_time=start_time,
                            tweet_fields = ["created_at", "text", "lang"],
                            user_fields = ['name', 'username', "location"],
                            sort_order=['relevancy'],
                            expansions='author_id',
                            max_results=100)
    for tweet,user in zip(tweets.data,tweets.includes['users']):
        tweet_info = {
        'created_at': tweet.created_at,
        'text': tweet.text,
        'source': tweet.source,
        'name': user.name,
        'username': user.username,
        'location': user.location}
        tweet_info_small_list.append(tweet_info)
    tweets_datasource = pd.DataFrame(tweet_info_small_list)
    tweets_datasource.to_csv('C:/home2/hzhx55/Dissertation/TwData_300.csv',sep=',', mode='a',encoding='utf_8')

In [19]:
#Get rid of the empty lines and sort all the data by the time it happened
raw_data=pd.read_csv('TwData_300.csv',sep=',',header=0,encoding='utf_8')
raw_data=raw_data.drop(columns=['Unnamed: 0','source'])
raw_data=raw_data[raw_data['text']!='text'].sort_values(by='created_at').reset_index(drop=True)

def treatment_marker(row):
    if row.created_at<'2019-10-01 00:00:00+00:00':
        answer='before covid'
    else:
        answer='after covid'
    return answer

raw_data['treatment/control']=raw_data.apply(treatment_marker,axis=1)

#Remove all the emoji used in the text
def emoji_free_text(text):
    pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FC"  # transport & map symbols
        u"\U0001F1E6-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u200a-\u200f"
        "]+", 
        flags = re.UNICODE)
    return pattern.sub(r'',text)
    
#Remove all the puntutation and number used in the text
def puntuation_free_text(text):
    tab=str.maketrans(dict.fromkeys(string.punctuation))
    pure_text=text.translate(tab)
    pure_text=''.join([let for let in pure_text if not let.isdigit()])
    return pure_text

def remove_meaningless_name(text):
    exclude_words_of_name=sw.words('english')
    new_name=[]
    for word in text.split():
        if word not in exclude_words_of_name:
            new_name.append(word)
    return ' '.join(new_name)

# Remove all the space used at the start of sentence, lower the case, and only need the first name of users
def remove_space(text):
    if len(text)==0:
        text='unknown'
    else:
        if text[0]==' ':
            text=text.lower()[1:]
        else:
            text=text.lower()
    return text

def remove_alpha(text):
    for word in text.split():
        if len(word)>1:
            pattern=re.compile(word)
            poistion=[match.start() for match in pattern.finditer(text)][0]
            text=text[poistion:]
            break
    return text

gender=['unknown']*len(raw_data.name)
fitered_name_list=[]
gender_list=[]

for name,sex,username in zip(raw_data.name,gender,raw_data.username):
    name=remove_alpha(remove_space(remove_meaningless_name(puntuation_free_text(emoji_free_text(name)))))
    if (name[0:3]=='sir'and len(name.split()[0])==3):
        name=name[4:]
        sex='male'
    elif (name[0:2]=='mr'and len(name.split()[0])==2):
        name=name[3:]
        sex='male'
    elif (name[0:2]=='dr'and len(name.split()[0])==2):
        name=name[3:]
        sex='male'
    elif (name[0:4]=='prof' and len(name.split()[0])==4):
        name=name[5:]
        sex='male'
    elif (name[0:4]=='miss' and len(name.split()[0])==4):
        name=name[5:]
        sex='female'
    elif (name[0:5]=='madam'and len(name.split()[0])==5):
        name=name[6:]
        sex='female'
    elif (name[0:2]=='ms'and len(name.split()[0])==2):
        name=name[3:]
        sex='female'
    elif (name[0:3]=='mrs'and len(name.split()[0])==3):
        name=name[4:]
        sex='female'
    elif (name[0:2]=='mx'and len(name.split()[0])==2):
       name=name[2:]
       sex='unknown'
    if len(name.split())==0:
        name='unknown'
    else:
        first_name = ''.join([word for word in name.split()[0]])
    if first_name!='unknown':
        first_name=first_name.capitalize()
    if len(first_name)<=2:
        first_name=remove_alpha(remove_meaningless_name(puntuation_free_text(emoji_free_text(username)))).capitalize()
    if len(re.findall('([A-Z][a-z]+)', first_name))!=0:
        first_name=re.findall('([A-Z][a-z]+)', first_name)[0]   
    if len(first_name)==1:
        first_name='unknown'
        pfint(firs)
    fitered_name_list.append(first_name)
    gender_list.append(sex)

raw_data['first name']=pd.DataFrame(fitered_name_list)
raw_data['gender']=pd.DataFrame(gender_list)
known_name=raw_data['first name'].loc[raw_data['gender'] =='unknown']
detector=gen.Detector()
raw_data['predicted gender']=raw_data['first name'].apply(lambda a: detector.get_gender(a))
raw_data['gender'].loc[(raw_data['gender']=='unknown')] = raw_data['predicted gender']

raw_data.loc[(raw_data['gender']=='mostly_female'),'gender']='female'
raw_data.loc[(raw_data['gender']=='mostly_male'),'gender']='male'
raw_data.loc[(raw_data['gender']=='andy'),'gender']='unknown'
raw_data=raw_data.drop(columns=['predicted gender'])

#Remove all the contact signs and people names mentioned inside the text
def free_contact(text):
    pattern = re.compile("@+")
    if len(re.findall(pattern,text))!=0:
        pattern = re.compile("@+")
        times=len([match.span() for match in pattern.finditer(text)])
        time=0
        while time<times:
            poistion=[match.start() for match in pattern.finditer(text)][0]
            text=text[:poistion]+text[poistion+len(text[poistion:].split()[0]):]
            text=text.replace("  ", " ")
            time+=1
        else:
            text=text
    return text

#Restore all emoji into readable meanings
def emoji_lemon(text):
    return emoji.demojize(text, delimiters=("", "")) 

def single_quote_transfer(text):
    text=re.sub("’ve"," have",text)
    text=re.sub("’d"," would like to",text)
    text=re.sub("n’t"," not",text)
    text=re.sub("’m"," am",text)
    text=re.sub("’s"," is",text)
    return text

new_processed_text=[]
for text in raw_data.text:
    pre_processed_text=remove_space(remove_meaningless_name(single_quote_transfer(puntuation_free_text(emoji_lemon(free_contact(text))))))
    new_processed_text.append(pre_processed_text)

raw_data.insert(loc=2, column='new_processed_text', value=new_processed_text)

In [20]:
print(len(raw_data.loc[raw_data['first name']=='unknown']))

0


In [21]:
raw_data.loc[raw_data['first name']=='unknown'][0:50]

,created_at,text,new_processed_text,name,username,location,treatment/control,first name,gender


In [240]:
s='Sea Witch'
print(len(re.findall('([A-Z][a-z]+)', s)),re.findall('([A-Z][a-z]+)', s)[0])


2 Sea


In [206]:
i=0
if i<1:
    i='a'
if i!='b':
    i='c'
i

'c'